In [1]:
import pandas as pd
import numpy as np
import qcew_2014_2021
# import all functions from qcew module
from qcew_2014_2021 import *


In [2]:
#FOR FOSSIL

fossil_industry_data = get_2014_industry_data('fossil')

cleaned_fossil_data = get_cleaned_2014_industry_data(fossil_industry_data)

fossil_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_fossil_data)

aggregated_fossil_data = aggregate_data_by_type(fossil_clean_with_geo,'fossil')



/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [5]:
aggregated_fossil_data

,area_fips,area_title,geographic_level,industry_code,year,annual_avg_emplvl
0,01000,Alabama -- Statewide,state,211,2014,711
1,01000,Alabama -- Statewide,state,211,2015,443
2,01000,Alabama -- Statewide,state,211,2016,438
3,01000,Alabama -- Statewide,state,211,2017,411
4,01000,Alabama -- Statewide,state,211,2018,411
...,...,...,...,...,...,...
94735,US000,U.S. TOTAL,nationwide,486,2017,50354
94736,US000,U.S. TOTAL,nationwide,486,2018,50474
94737,US000,U.S. TOTAL,nationwide,486,2019,51522
94738,US000,U.S. TOTAL,nationwide,486,2020,51818


In [12]:
#FOR TOTALS

total_employment_data = get_2014_industry_data('total')

cleaned_total_data = get_cleaned_2014_industry_data(total_employment_data)

total_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_total_data)

aggregated_total_data = aggregate_data_by_type(total_clean_with_geo,'total')

# drop ownership and industry column

aggregated_total_data = aggregated_total_data.drop(columns=['own_code','industry_code'])



/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [9]:
# bring together

total_fossil_data = aggregated_fossil_data.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_data = total_fossil_data.rename(columns={'annual_avg_emplvl':'fossil_employees'})

total_fossil_data

,area_fips,area_title,geographic_level,year,fossil_employees
0,01000,Alabama -- Statewide,state,2014,19327
1,01000,Alabama -- Statewide,state,2015,18179
2,01000,Alabama -- Statewide,state,2016,16866
3,01000,Alabama -- Statewide,state,2017,17590
4,01000,Alabama -- Statewide,state,2018,16851
...,...,...,...,...,...
28068,US000,U.S. TOTAL,nationwide,2017,1072244
28069,US000,U.S. TOTAL,nationwide,2018,1149090
28070,US000,U.S. TOTAL,nationwide,2019,1161099
28071,US000,U.S. TOTAL,nationwide,2020,990214


In [13]:
aggregated_total_data = aggregated_total_data.rename(columns={'annual_avg_emplvl':'total_employees'})
aggregated_total_data

,area_fips,area_title,geographic_level,year,total_employees
0,01000,Alabama -- Statewide,state,2014,1863561
1,01000,Alabama -- Statewide,state,2015,1890340
2,01000,Alabama -- Statewide,state,2016,1915306
3,01000,Alabama -- Statewide,state,2017,1936819
4,01000,Alabama -- Statewide,state,2018,1961625
...,...,...,...,...,...
153948,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2017,19449252
153949,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2018,19662669
153950,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2019,19771027
153951,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2020,18904204


In [15]:
# merge on year

combined_attempt = aggregated_total_data.merge(total_fossil_data,on=['area_fips','area_title','geographic_level','year'],how='left')

In [18]:
combined_attempt['percent_of_fossil_employment'] = combined_attempt['fossil_employees'] / combined_attempt ['total_employees'] * 100

In [22]:
# criteria only relevant for MSAs and (some counties)

fossil_employment_ratios = combined_attempt.query("geographic_level== 'metropolitan_stat_area' | geographic_level == 'county'")

In [27]:
fossil_employment_ratios[fossil_employment_ratios['percent_of_fossil_employment'] > 0.17]

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees,percent_of_fossil_employment
88,01003,"Baldwin County, Alabama",county,2014,65024,251.0,0.386011
89,01003,"Baldwin County, Alabama",county,2015,67585,252.0,0.372864
90,01003,"Baldwin County, Alabama",county,2016,70107,255.0,0.363730
91,01003,"Baldwin County, Alabama",county,2017,71999,240.0,0.333338
92,01003,"Baldwin County, Alabama",county,2018,74051,271.0,0.365964
...,...,...,...,...,...,...,...
152493,C4974,"Yuma, AZ MSA",metropolitan_stat_area,2018,9103,60.0,0.659123
152494,C4974,"Yuma, AZ MSA",metropolitan_stat_area,2019,9121,61.0,0.668786
152495,C4974,"Yuma, AZ MSA",metropolitan_stat_area,2020,8614,62.0,0.719759
152496,C4974,"Yuma, AZ MSA",metropolitan_stat_area,2021,8567,58.0,0.677016
